<a href="https://colab.research.google.com/github/x1001000/sed-yamnet-raspberrypi/blob/main/colab_notebooks/%E9%9B%A8%E8%81%B2_WFH_Day_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 瀏覽貼文

In [1]:
from IPython.display import HTML
HTML('<iframe src="https://www.facebook.com/plugins/video.php?height=314&href=https%3A%2F%2Fwww.facebook.com%2Ftsaiingwen%2Fvideos%2F1052822178577809%2F&show_text=true&width=560" width="560" height="429" style="border:none;overflow:hidden" scrolling="no" frameborder="0" allowfullscreen="true" allow="autoplay; clipboard-write; encrypted-media; picture-in-picture; web-share" allowFullScreen="true"></iframe>')

# 安裝套件、下載影片

In [2]:
!pip install -q soundfile
!pip install -q pydub
!pip install -q fb-down
!fbdown https://www.facebook.com/tsaiingwen/videos/1052822178577809 --output 20210524.mp4
!git clone -q https://github.com/x1001000/raspberrypi3-yamnet-sed
!cp raspberrypi3-yamnet-sed/yamnet/* .


                                                              
,------.,-----.      ,------.   ,-----. ,--.   ,--.,--.  ,--. 
|  .---'|  |) /_     |  .-.  \ '  .-.  '|  |   |  ||  ,'.|  | 
|  `--, |  .-.  \    |  |  \  :|  | |  ||  |.'.|  ||  |' '  | 
|  |`   |  '--' /    |  '--'  /'  '-'  '|   ,'.   ||  | `   | 
`--'    `------'     `-------'  `-----' '--'   '--'`--'  `--' 
                   By : GURKIRAT SINGH
            https://tbhaxor.github.com/fbdown    
    
URL : https://www.facebook.com/tsaiingwen/videos/1052822178577809
Save As : 20210524.mp4
Video Size :  785.04 KB

786KB [00:00, 13191.22KB/s]           
Download Complete !!!


# 匯入套件、聲音辨識模型

In [3]:
import numpy as np
import resampy
import soundfile as sf
import tensorflow as tf

import params as yamnet_params
import yamnet as yamnet_model

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

from pydub import AudioSegment

from IPython.display import display, Audio
from time import sleep

params = yamnet_params.Params()
yamnet = yamnet_model.yamnet_frames_model(params)
yamnet.load_weights('yamnet.h5')
yamnet_classes = yamnet_model.class_names('yamnet_class_map_zh-tw.csv')

# 擷取聲音

In [4]:
AudioSegment.from_file('20210524.mp4').export('20210524.wav', format='wav')
# https://github.com/tensorflow/models/blob/master/research/audioset/yamnet/inference.py#L40
wav_data, sr = sf.read('20210524.wav', dtype=np.int16)
assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
waveform = wav_data / tf.int16.max#32768.0  # Convert to [-1.0, +1.0]
waveform = waveform.astype('float32')

# Convert to mono and the sample rate expected by YAMNet.
if len(waveform.shape) > 1:
    waveform = np.mean(waveform, axis=1)
if sr != params.sample_rate:
    waveform = resampy.resample(waveform, sr, params.sample_rate)

# 執行推論

In [5]:
MA = 2
SR = int(params.sample_rate)
for i in range(MA, len(waveform)//SR):
    scores, embeddings, spectrogram = yamnet(waveform[(i-MA)*SR:i*SR])
    prediction = np.mean(scores[:-1], axis=0)
    top5 = np.argsort(prediction)[::-1][:5]
    print(f'{i%60:2d}s',
        ''.join(f" {prediction[i]:.2f} 👉{yamnet_classes[i][:10].ljust(10, '　')}" for i in top5))

 2s  0.16 👉車輛　　　　　　　　 0.10 👉室外，城市或人造　　 0.09 👉火　　　　　　　　　 0.08 👉機動車輛（道路）　　 0.08 👉汽車　　　　　　　　
 3s  0.37 👉水　　　　　　　　　 0.23 👉雨　　　　　　　　　 0.15 👉雨滴　　　　　　　　 0.10 👉倒水　　　　　　　　 0.10 👉液體　　　　　　　　
 4s  0.59 👉水　　　　　　　　　 0.59 👉雨　　　　　　　　　 0.39 👉雨滴　　　　　　　　 0.36 👉雨表面上　　　　　　 0.17 👉油炸（食品）　　　　
 5s  0.51 👉水　　　　　　　　　 0.48 👉雨　　　　　　　　　 0.31 👉雨滴　　　　　　　　 0.25 👉雨表面上　　　　　　 0.11 👉油炸（食品）　　　　
 6s  0.29 👉水　　　　　　　　　 0.23 👉雨　　　　　　　　　 0.17 👉雨滴　　　　　　　　 0.13 👉言語　　　　　　　　 0.12 👉雨表面上　　　　　　
 7s  0.14 👉水　　　　　　　　　 0.11 👉動物　　　　　　　　 0.11 👉嘶　　　　　　　　　 0.10 👉液體　　　　　　　　 0.09 👉蒸汽　　　　　　　　
 8s  0.40 👉水　　　　　　　　　 0.19 👉雨　　　　　　　　　 0.13 👉液體　　　　　　　　 0.12 👉雨滴　　　　　　　　 0.08 👉油炸（食品）　　　　
 9s  0.46 👉水　　　　　　　　　 0.29 👉雨　　　　　　　　　 0.27 👉雨滴　　　　　　　　 0.18 👉液體　　　　　　　　 0.15 👉雨表面上　　　　　　
10s  0.34 👉雨　　　　　　　　　 0.30 👉水　　　　　　　　　 0.22 👉雨滴　　　　　　　　 0.19 👉雨表面上　　　　　　 0.09 👉室外，農村或天然　　
11s  0.44 👉雨　　　　　　　　　 0.37 👉雨滴　　　　　　　　 0.33 👉水　　　　　　　　　 0.23 👉雨表面上　　　　　　 0.09 👉室外，農村或天然　　
12s  0.43 👉雨　　　　　　　　　 0.36 👉雨滴　　　　　　　　 0.31 👉雨表面上　　　　　　 0.29 👉水　　　　　　　　　 0.08 👉沙沙　　　　　　　　
13s  0.65 

## 👉左側數字為👉右側聲音事件的可能性，因為一段聲音同時包含多種可能事件，所以數字之間並沒有總和等於1的關係！